In [ ]:
## このプログラムは「Dogs vs. Cats Redux: Kernels Edition」において
## 作成したノートブックで動作します

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# 訓練データとテストデータの解凍
import os, shutil, zipfile

# 解凍するzipファイル名
data = ['train', 'test']

# train.zip、test.zipをカレントディレクトリに展開
for el in data:
    with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/' + el + ".zip", "r") as z:
        z.extractall()

In [ ]:
# データの前処理
import pandas as pd

# trainフォルダー内のファイル名を取得してfilenamesに格納
filenames = os.listdir("./train")
# store the label for each image file
categories = []

# 訓練データのファイル名のdog.x.jpg、cat.x.jpgを使って1と0のラベルを生成
for filename in filenames:
    # ファイル名を分割して先頭要素(dog/cat)のみを取り出し、
    # dogは1、catは0をラベルにしてcategoryに格納
    category = filename.split('.')[0]
    if category == 'dog':
        # dogならラベル1として追加
        categories.append(1)
    else:
        # catならラベル0として追加
        categories.append(0)

# dfの列filenameにファイル名filenamesを格納
# 列categoriesにラベルの値categoryを格納
df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

# データフレームの先頭から5行目までを出力
df.head()

In [ ]:
# dog(1)とcat(0)の総数をグラフにする
df['category'].value_counts().plot.bar()

In [ ]:
# ランダムに選んだ12枚の画像を出力する
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import random
%matplotlib inline

# ランダムに16枚取り出す
sample = random.sample(filenames, 16)

# 描画するエリアのサイズは12×12
plt.figure(figsize=(12, 12))

for i in range(0, 16):
    # 4×4のマス目の左上隅から順番に描画
    plt.subplot(4, 4, i+1)
    # sampleに格納されたi番目の画像
    fname = sample[i]
    # trainフォルダーから画像を読み込む
    image = load_img("./train/"+fname)
    # 画像を描画
    plt.imshow(image)
    plt.axis('off') # 目盛りは非表示
plt.tight_layout()
plt.show()

In [ ]:
# 訓練データを訓練用と検証用に分ける
from sklearn.model_selection import train_test_split

# 訓練データの総数25000の10%を検証データにする
train_df, validate_df = train_test_split(df, test_size=0.1)
# 行インデックスを振り直す
train_df = train_df.reset_index()
validate_df = validate_df.reset_index()

# 訓練データの数を取得
total_train = train_df.shape[0]
# 検証データの数を取得
total_validate = validate_df.shape[0]

# 訓練、検証データの数を出力
print(total_train)
print(total_validate)

In [ ]:
# 訓練データを加工処理する
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 画像をリサイズするときのサイズ
img_width, img_height = 224, 224
target_size = (img_width, img_height)
# ミニバッチのサイズ
batch_size = 16

# データフレームに格納したファイル名の列名とラベルの列名
x_col, y_col = 'filename', 'category'
# flow_from_dataframe()で画像を生成する際のclass_modeオプションの値
# ジェネレーターが返すラベルの配列の形状として二値分類の'binary'を格納
class_mode = 'binary'

# 画像を加工するジェネレーターを生成
train_datagen = ImageDataGenerator(
    rescale=1./255,         # RGB値を0～1.0の範囲に変換
    rotation_range=15,      # ランダムに回転
    shear_range=0.2,        # シアー変換
    zoom_range=0.2,         # 拡大
    horizontal_flip=True,   # 水平方向に反転
    width_shift_range=0.1,  # 平行移動
    height_shift_range=0.1  # 垂直移動
)


# flow_from_dataframe()の引数class_mode = "binary"の場合、
# ラベルが格納されたtrain_dfのy_col = 'category'の列の値は
# 文字列であることが必要なので、1と0の数値を文字列に変換しておく
train_df['category'] = train_df['category'].astype(str) 

# ジェネレータで加工した画像の生成
train_generator = train_datagen.flow_from_dataframe(
    train_df,    # 訓練用のデータフレーム
    "./train/",  # 画像データのディレクトリ
    x_col=x_col, # ファイル名が格納された列
    y_col=y_col, # ラベルが格納された列 (文字列に変換済み)
    class_mode=class_mode, # ラベルの配列の形状
    target_size=target_size, # 画像のサイズ
    batch_size=batch_size    # ミニバッチのサイズ
)

In [ ]:
# 検証データを加工処理する

# 画像を加工するジェネレーターを生成
# データ拡張は必要ないのでRGB値の変換のみを行う
validation_datagen = ImageDataGenerator(rescale=1./255)

# flow_from_dataframe()の引数class_mode = "binary"の場合、
# ラベルが格納されたvalidate_dfのy_col = 'category'の列の値は
# 文字列であることが必要なので、1と0の数値を文字列に変換しておく
validate_df['category'] = validate_df['category'].astype(str)

# ジェネレータで加工した画像の生成
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,  # 検証用のデータフレーム
    "./train/",   # 画像データのディレクトリ
    x_col=x_col,  # ファイル名が格納された列
    y_col=y_col,  # ラベルが格納された列(文字列に変換済み)
    class_mode=class_mode,   # ラベルの配列の形状
    target_size=target_size, # 画像のサイズ
    batch_size=batch_size    # ミニバッチのサイズ
)

In [ ]:
# 訓練データから1サンプル取り出し、加工処理後の9パターンを表示

# 訓練データから1サンプル取り出し、reset_index()でインデックスを振り直す
# drop=Trueは元のインデックスを削除するためのもの
example_df = train_df.sample(n=1).reset_index(drop=True)
# DataFrameIteratorオブジェクトを生成
example_generator = train_datagen.flow_from_dataframe(
    example_df,       # サンプルデータを格納したデータフレーム
    "./train/",       # 画像データの場所
    x_col='filename', # ファイル名の列名
    y_col='category', # 正解ラベルの列名
    target_size=target_size # 画像をリサイズする
)
# 描画エリアのサイズは12×12
plt.figure(figsize=(12, 12))
# 加工処理後の9パターンを表示
for i in range(0, 9):
    # 3×3のマス目の左上隅から順番に描画
    plt.subplot(3, 3, i+1)
    for X_batch, Y_batch in example_generator:
        # X_batchの1つ目の画像データを抽出
        image = X_batch[0]
        # 抽出した画像を描画したらbreakする
        plt.imshow(image)
        break
plt.show()

In [ ]:
# 3層の畳み込み層を持つCNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

# Sequentualオブジェクトを生成
model = Sequential()

# 入力データの形状
input_shape = (img_width, img_height, 3)

# 第1層:畳み込み層1
model.add(
    Conv2D(
        filters=32,              # フィルターの数は32
        kernel_size=(3, 3),      # 3×3のフィルターを使用
        padding='same',          # ゼロパディングを行う
        activation='relu',       # 活性化関数はReLU
        input_shape=input_shape, # 入力データの形状
        ))

# 第2層:プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト25％
model.add(Dropout(0.25))

# 第3層:畳み込み層2
model.add(
    Conv2D(
        filters = 64,        # フィルターの数は32
        kernel_size = (3,3), # 3×3のフィルターを使用
        padding='same',      # ゼロパディングを行う
        activation='relu',   # 活性化関数はReLU
        ))

# 第4層:プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト25％
model.add(Dropout(0.25))

# 第5層:畳み込み層3
model.add(
    Conv2D(
        filters=128,          # フィルターの数は64
        kernel_size=(3, 3),   # 3×3のフィルターを使用
        padding='same',       # ゼロパディングを行う
        activation='relu',    # 活性化関数はReLU
        ))

# 第6層:プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2))
)
# ドロップアウト25％
model.add(Dropout(0.25))

# (batch_size, rows, cols, channels)の4階テンソルに
# プーリング演算適用後、(batch_size, channels)の2階テンソルにフラット化
model.add(
    GlobalMaxPooling2D())

# 第7層
model.add(
    Dense(128,                   # ユニット数128
          activation='relu'))    # 活性化関数はReLU
# ドロップアウト25％
model.add(Dropout(0.25))

# 第8層:出力層
model.add(
    Dense(1,                     # ニューロン数は1個
          activation='sigmoid')) # 活性化関数はSigmoid

# モデルのコンパイル
model.compile(
    loss='binary_crossentropy',    # バイナリ用のクロスエントロピー誤差
    metrics=['accuracy'],          # 学習評価として正解率を指定
    optimizer=optimizers.RMSprop() # RMSpropで最適化
)

In [ ]:
# 学習を行う

import math
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, Callback

# 学習率をスケジューリングする
def step_decay(epoch):
    initial_lrate = 0.001 # 学習率の初期値
    drop = 0.5            # 減衰率は50%
    epochs_drop = 10.0    # 10エポック毎に減衰する
    lrate = initial_lrate * math.pow(
        drop,
        math.floor((epoch)/epochs_drop)
    )
    return lrate

# 学習率のコールバック
lrate = LearningRateScheduler(step_decay)
 
# 学習の進捗を監視して早期終了するコールバック
earstop = EarlyStopping(
    monitor='val_loss', # 監視対象は損失
    min_delta=0,        # 改善として判定される最小変化値
    patience=5)         # 改善が見られないと判断されるエポック数を5に拡大


# 学習の実行
# GPU使用による所要時間:
epochs = 40        # エポック数
history = model.fit(
    # 訓練データ
    train_generator,
    # エポック数
    epochs=epochs,
    # 訓練時のステップ数
    steps_per_epoch = total_train//batch_size,
    # 検証データ
    validation_data=validation_generator,
    # 検証時のステップ数
    validation_steps = total_validate//batch_size,
    # 学習の進捗状況を出力する    
    verbose=1,
    # 学習率のスケジューラーとアーリーストッピングをコール
    callbacks=[lrate, earstop]
)